In [2]:
import os
import sys
import logging
import settings


In [3]:
stock_code = '005930'  # 삼성전자

In [4]:
# 로그 기록
log_dir = os.path.join(settings.BASE_DIR, 'logs/%s' % stock_code)
timestr = settings.get_time_str()
if not os.path.exists('logs/%s' % stock_code):
    os.makedirs('logs/%s' % stock_code)
file_handler = logging.FileHandler(filename=os.path.join(
    log_dir, "%s_%s.log" % (stock_code, timestr)), encoding='utf-8')
stream_handler = logging.StreamHandler(sys.stdout)
file_handler.setLevel(logging.DEBUG)
stream_handler.setLevel(logging.INFO)
logging.basicConfig(format="%(message)s",
                    handlers=[file_handler, stream_handler], level=logging.DEBUG)

In [5]:
import pandas as pd

csv_dt = pd.read_csv(os.path.join(settings.BASE_DIR,'data/chart_data/{}.csv'.format(stock_code)))
csv_dt.head()

,1985-01-04,"6,511","6,511.1","6,473","6,473.1","2,232"
0,1985-01-05,"6,434","6,465","6,427","6,427","2,167"
1,1985-01-07,"6,434","6,511","6,427","6,442","15,417"
2,1985-01-08,"6,434","6,434","6,358","6,358","16,879"
3,1985-01-09,"6,312","6,312","6,090","6,136","6,488"
4,1985-01-10,"6,189","6,274","6,189","6,205","9,242"


In [8]:
import data_manager
from policy_learner import PolicyLearner

In [12]:
    # 주식 데이터 준비

os.path.join(settings.BASE_DIR,'data/chart_data/{}.csv'.format(stock_code))


'D:\\ML\\rltrader\\data/chart_data/005930.csv'

In [13]:
#ohlc
chart_data = data_manager.load_chart_data(os.path.join(settings.BASE_DIR,'data/chart_data/{}.csv'.format(stock_code)))
chart_data.head()

,date,open,high,low,close,volume
0,1985-01-04,6511,6511,6473,6473,2232
1,1985-01-05,6434,6465,6427,6427,2167
2,1985-01-07,6434,6511,6427,6442,15417
3,1985-01-08,6434,6434,6358,6358,16879
4,1985-01-09,6312,6312,6090,6136,6488


In [14]:
#이동평균
prep_data = data_manager.preprocess(chart_data)
prep_data.head()

,date,open,high,low,close,volume,close_ma5,volume_ma5,close_ma10,volume_ma10,close_ma20,volume_ma20,close_ma60,volume_ma60,close_ma120,volume_ma120
0,1985-01-04,6511,6511,6473,6473,2232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-01-05,6434,6465,6427,6427,2167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-01-07,6434,6511,6427,6442,15417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-01-08,6434,6434,6358,6358,16879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-01-09,6312,6312,6090,6136,6488,6367.2,8636.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#주가와 거래양의 비율(에이전트의 상태)
training_data = data_manager.build_training_data(prep_data)
training_data.head()

,date,open,high,low,close,volume,close_ma5,volume_ma5,close_ma10,volume_ma10,close_ma20,volume_ma20,close_ma60,volume_ma60,close_ma120,volume_ma120,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,close_ma5_ratio,volume_ma5_ratio,close_ma10_ratio,volume_ma10_ratio,close_ma20_ratio,volume_ma20_ratio,close_ma60_ratio,volume_ma60_ratio,close_ma120_ratio,volume_ma120_ratio
0,1985-01-04,6511,6511,6473,6473,2232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.005871,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-01-05,6434,6465,6427,6427,2167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006025,0.005913,0.000000,-0.007106,-0.029122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-01-07,6434,6511,6427,6442,15417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001089,0.010711,-0.002328,0.002334,6.114444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-01-08,6434,6434,6358,6358,16879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001242,0.011953,0.000000,-0.013039,0.094830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-01-09,6312,6312,6090,6136,6488,6367.2,8636.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007235,0.028683,-0.007497,-0.034917,-0.615617,-0.036311,-0.248778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
len(training_data.index)

8826

In [18]:
    # 기간 필터링
training_data = training_data[(training_data['date'] >= '2017-01-01') &
                                  (training_data['date'] <= '2017-12-31')]
len(training_data.index)
    

243

In [19]:
training_data = training_data.dropna()
len(training_data.index)

243

In [22]:
len(training_data.columns)

31

In [24]:
# 차트 데이터 분리
features_chart_data = ['date', 'open', 'high', 'low', 'close', 'volume']
chart_data = training_data[features_chart_data]

In [28]:
chart_data.head() 

(243,             date     open     high      low    close  volume
 8551  2017-01-02  1799000  1812000  1794000  1805000   93012
 8552  2017-01-03  1814000  1831000  1801000  1824000  147153
 8553  2017-01-04  1825000  1826000  1805000  1808000  159435
 8554  2017-01-05  1803000  1803000  1777000  1778000  219349
 8555  2017-01-06  1809000  1822000  1802000  1810000  177619)

In [29]:
len(chart_data.index)

243

In [30]:
# 학습 데이터 분리
features_training_data = [
        'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
        'close_lastclose_ratio', 'volume_lastvolume_ratio',
        'close_ma5_ratio', 'volume_ma5_ratio',
        'close_ma10_ratio', 'volume_ma10_ratio',
        'close_ma20_ratio', 'volume_ma20_ratio',
        'close_ma60_ratio', 'volume_ma60_ratio',
        'close_ma120_ratio', 'volume_ma120_ratio'
    ]
training_data = training_data[features_training_data]


In [31]:
len(training_data.columns)

15

In [33]:
training_data.shape , training_data.shape[1] 

((243, 15), 15)

In [35]:
    def _get_batch(self, memory, batch_size, discount_factor, delayed_reward):
        num_features = 17
        x = np.zeros((batch_size, 1, num_features))
        y = np.full((batch_size, 2), 0.5)

        for i, (sample, action, reward) in enumerate(
                reversed(memory[-batch_size:])):
            x[i] = np.array(sample).reshape((-1, 1, num_features))
            y[i, action] = (delayed_reward + 1) / 2
            if discount_factor > 0:
                y[i, action] *= discount_factor ** i
        return x, y

In [38]:
discount_factor=0
batch_size = 0
delayed_reward = 0
memory = [(
                    memory_sample[i],
                    memory_action[i],
                    memory_reward[i])
                    for i in list(range(len(memory_action)))[-max_memory:]
                ]

NameError: name 'memory_action' is not defined

In [39]:
x, y = _get_batch(memory, batch_size, discount_factor, delayed_reward)

NameError: name 'memory' is not defined

# #리스트 내포(List comprehension)

In [44]:
[(1,2) for i in list(range(5))[-60:]]

[(1, 2), (1, 2), (1, 2), (1, 2), (1, 2)]

In [45]:
[(1,2) for i in list(range(5))]

[(1, 2), (1, 2), (1, 2), (1, 2), (1, 2)]

In [46]:
gugudan = [x*y for x in range(2,10)
             for y in range(1,10)]
gugudan

[2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 3,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 4,
 8,
 12,
 16,
 20,
 24,
 28,
 32,
 36,
 5,
 10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 6,
 12,
 18,
 24,
 30,
 36,
 42,
 48,
 54,
 7,
 14,
 21,
 28,
 35,
 42,
 49,
 56,
 63,
 8,
 16,
 24,
 32,
 40,
 48,
 56,
 64,
 72,
 9,
 18,
 27,
 36,
 45,
 54,
 63,
 72,
 81]